In [206]:
import pandas as pd
import os
import numpy as np
import re
import datetime

In [207]:
ref_df = pd.read_csv('which_race.csv', header=0)
csv_list = os.listdir('clean_frames')
counter = 34

In [241]:
d2 = {}
d3 = {}
for race_id in range(counter):
    d2['race_df_'+str(race_id)]=pd.read_csv('clean_frames/race_df_'+str(race_id)+'.csv')
    d3['pit_stop_df_'+str(race_id)]=pd.read_csv('clean_frames/pit_stop_df_'+str(race_id)+'.csv')

In [242]:
race_df = d2['race_df_0']
# dfone = pd.concat(d2.values())
pit_stop_df = d3['pit_stop_df_0']

In [243]:
race_df['TIME'] = race_df['TIME'].apply(lambda x: int(x.split(':')[0])*60 + float(x.split(':')[1]))

In [244]:
race_df['cumtime']=race_df[['TIME','NO']].groupby('NO').cumsum(skipna=True)

In [248]:
pit_stop_df.columns

Index(['NO', 'DRIVER', 'ENTRANT', 'LAP', 'TIME OF DAY', 'STOP', 'DURATION',
       'TOTAL TIME'],
      dtype='object')

In [231]:
race_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [247]:
pit_stop_df.drop(columns = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 6'], inplace=True)

In [249]:
def distances(x):
    return x.values[:,None]-x.values

In [250]:
# pita_df = pd.DataFrame(distances(race_df['TIME'][race_df['LAP']==1]),columns=list(race_df['NO'][race_df['LAP']==1].values),index = race_df['NO'][race_df['LAP']==1].values)

# pita_df['LAP']=1

# pita_df.reset_index(level=0, inplace=True)

# pita_df.rename({'index':'NO'},axis='columns',inplace=True)

In [251]:
# df1 = race_df.merge(pita_df, on=['LAP','NO'], how="outer")

In [252]:
# pita_df2 = pd.DataFrame(distances(race_df['TIME'][race_df['LAP']==2]),columns=list(race_df['NO'][race_df['LAP']==2].values),index = race_df['NO'][race_df['LAP']==2].values)

# pita_df2['LAP'] = 2

# pita_df2.reset_index(level=0, inplace=True)

# pita_df2.rename({'index':'NO'},axis='columns',inplace=True)

In [253]:
# pit_stop_df.set_index(['NO','LAP'], inplace=True)
# new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')

In [254]:
last_lap = max(race_df['LAP'])

In [257]:
d_gaps={}
for i in range(0,last_lap):
    lap = i+1
    gaps_df = pd.DataFrame(distances(race_df['TIME'][race_df['LAP']==lap]),columns=list(race_df['NO'][race_df['LAP']==lap ].values),index = race_df['NO'][race_df['LAP']==lap].values)

    gaps_df['LAP'] = lap

    gaps_df.reset_index(level=0, inplace=True)

    gaps_df.rename({'index':'NO'},axis='columns',inplace=True)  

#     gaps_df.set_index(['NO','LAP'], inplace=True)
    
    d_gaps['gaps_df_lap_'+str(lap)] = gaps_df
    
#     gaps_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [258]:
race_df.set_index(['NO','LAP'], inplace=True)
for i in range(0,last_lap):
    lap = i+1
    gaps_df = d_gaps['gaps_df_lap_'+str(lap)]    
    gaps_df.set_index(['NO','LAP'], inplace=True)
    if lap==1:
        race_df = race_df.merge(gaps_df, on=['NO','LAP'], how="outer")
    else:
        race_df.update(gaps_df, join='left')#, overwrite=True, filter_func=None, raise_conflict=False)

In [ ]:
race_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [261]:
gaps_df.columns#.drop(columns = ['Unnamed: 0'], inplace=True)

Index([6, 22, 44, 5, 12, 3, 19, 8, 27, 77, 55, 33, 30, 20, 94, 11], dtype='object')

In [651]:
new_df = race_df.merge(pit_stop_df, on = ['NO','LAP'], how ='outer')
# new_df.drop(columns = ['Unnamed: 0','Unnamed: 0'], inplace=True)

In [652]:
new_df.set_index(['NO','LAP'], inplace=True)

In [653]:
# new_df['stop_count'] = np.nan
new_df['stop_count'] = new_df['STOP']
###filling the first lap stop_count0s
new_df['stop_count'][(new_df.index.get_level_values(1)==1) & (new_df['STOP']!=1)] = 0

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [658]:
x = pd.concat([new_df['stop_count'][new_df.index.get_level_values(0)==19].unstack(0).ffill().stack(0)],axis=1).swaplevel();
x.columns = ['stop_count'];
new_df.update(x,join='left');
# new_df['stop_count'][new_df.index.get_level_values(0)==19]

In [484]:
# new_df['STOP'].update(new_df['STOP'].groupby(level=0).ffill())

In [494]:
# new_df.update(new_df.groupby(level=0).ffill())
# df = df.groupby(level='group').bfill()
# df = df.unstack(0).bfill().stack().swaplevel(0, 1).reindex_like(df)
# # Column specific
# df.Value = df.groupby(level=0).Value.bfill()

In [461]:
# new_df['d_stop']=0
# new_df['d_stop'][new_df['STOP']>0]=1

In [656]:
# for _n in new_df.index.get_level_values(0).unique():
#     print(new_df[new_df.index.get_level_values(0)==_n]['STOP'])

In [302]:
# for x in d_gaps.keys():
#      print(d_gaps[x].shape)
# d_gaps['gaps_df_lap_1'].NO.apply(type)
# d_gaps['gaps_df_lap_1'].LAP.apply(type)

In [ ]:
# how do i cycle through laps without loop? can i vectorize?

# how do i impute nan's for running and RF? choose a time which should not matter based on intuition

# exclude teammate from traffic when feature engineering

# tuesday it is 3 minutes (8-10 quick slides)

# wednesday science fair 
'''there will be a screen on a table
CONTEXT talk about the relevance of the problem 10%
PROCESS general strokes of what the model and why i chose to do it like that 40%+
RESULTS results = metrics (predictive power, importance, inference?) 40%+
NEXT STEPS 10%
 '''

